In [ ]:
#
# 2024-04-13 // no longer works correctly... unsure if it's because of recent changes
#               or do to the change in multiple df's vs single df
#
import pandas as pd
import networkx as nx
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *

In [ ]:
rt = RACETrack()

In [ ]:
nf_df_orig        = pd.read_csv("../../netflow_sample.csv")
nf_df_orig['beg'] = nf_df_orig['beg'].astype('datetime64[ns]')

In [ ]:
df10k = nf_df_orig.sample(10000)

relates=[('sip','dip')]
g_nx = rt.createNetworkXGraph(df10k,relates)
pos  = nx.spring_layout(g_nx)

In [ ]:
rt.linkNode(df10k, relates, pos, 
            node_shape=rt.shapeByDataFrameLength,
            draw_labels=False, w=512, h=512)

In [ ]:
def dbytBuckets(x):
    if x   < 100:
        return '< 100'
    elif x < 1000:
        return '< 1k'
    elif x < 10000:
        return '< 10k'
    else:
        return '> 10k'
nf_df_orig['dbyt_bucket'] = nf_df_orig['DBYT'].apply(lambda x: dbytBuckets(x))

rt.co_mgr.str_to_color_lu['< 100']  = '#ff0000'
rt.co_mgr.str_to_color_lu['< 1k']   = '#9090ff'
rt.co_mgr.str_to_color_lu['< 10k']  = '#0000ff'
rt.co_mgr.str_to_color_lu['> 10k']  = '#000000'

_df = pd.DataFrame(nf_df_orig.groupby('dip').size()) \
      .reset_index() \
      .rename({0:'rows'},axis=1) \
      .query('rows > 1000')

dips_to_keep = set(_df['dip'])
# dips_to_keep = set(['122.1.254.10'])

df_dips = nf_df_orig.query('dip in @dips_to_keep')
len(df_dips)

In [ ]:
df_dips_agg = rt.temporalStatsAggregationWithGBFields(df_dips, fields='DBYT', gb_fields=['dip','dbyt_bucket'], freq='D', fill_missing=True)
df_dips_minus_time = df_dips.drop('beg',axis=1)

In [ ]:
g_nx = rt.createNetworkXGraph(df_dips_minus_time,relates)
pos  = nx.spring_layout(g_nx)
#pos['122.1.254.10'] = [100,100]
#pos['192.168.1.2']  = [200,200]
#pos['192.168.1.7']  = [100,200]

In [ ]:
df = pd.concat([df_dips_minus_time,df_dips_agg])
_svg = rt.linkNode(df, relates, pos, color_by='dbyt_bucket',
                   node_shape='small_multiple',
                   sm_type='xy', sm_params={'x_field':'beg','y_field':'DBYT_mean','line_groupby_field':'dbyt_bucket','dot_size':None}, 
                   sm_w=64, sm_h=64, 
                   draw_labels=False, w=1024, h=1024)
_svg._repr_svg_()

In [ ]:
rt.displaySVGAsImage(_svg)

In [ ]:
_svg

In [ ]:

rt.displaySVGAsImage(rt.linkNode([df_dips,df_dips_agg], relates, pos, draw_labels=True, link_shape='curve', link_arrow=True, w=1024, h=1024))

In [ ]:
_w_sm_override,_h_sm_override = 256,128

df_dips_agg = rt.temporalStatsAggregationWithGBFields(df_dips, fields='DBYT', gb_fields=['dip','dbyt_bucket'], freq='D', fill_missing=True)
category_dip_to_sm = {}
rt.smallMultiples(df_dips_agg, category_by='dip', color_by='dbyt_bucket', show_df_multiple=False, category_to_sm=category_dip_to_sm,
                  sm_type='xy', w_sm_override=_w_sm_override, h_sm_override=_h_sm_override,
                  sm_params={'x_field':'beg', 'y_field':'DBYT_median', 'line_groupby_field':'dbyt_bucket', 'line_groupby_w':2, 'dot_size':None, 'draw_labels':False})
df_sips_agg = rt.temporalStatsAggregationWithGBFields(df_dips, fields='DBYT', gb_fields=['sip','dbyt_bucket'], freq='D', fill_missing=True)
category_sip_to_sm = {}
rt.smallMultiples(df_sips_agg, category_by='sip', color_by='dbyt_bucket', show_df_multiple=False, category_to_sm=category_sip_to_sm,
                  sm_type='xy', w_sm_override=_w_sm_override, h_sm_override=_h_sm_override,
                  sm_params={'x_field':'beg', 'y_field':'DBYT_median', 'line_groupby_field':'dbyt_bucket', 'line_groupby_w':2, 'dot_size':None, 'draw_labels':False})

ip_to_svg = {}
for x in category_dip_to_sm.keys():
    ip_to_svg[x] = category_dip_to_sm[x]
for x in category_sip_to_sm.keys():
    ip_to_svg[x] = category_sip_to_sm[x]

rt.displaySVG(rt.linkNode(df_dips, relates, pos, bounds_percent=0.1, draw_labels=True, link_shape='curve', link_arrow=True, node_shape=ip_to_svg, w=1600, h=1600))

In [ ]:
rt.displaySVGAsImage(
    rt.smallMultiples(df_sips_agg, category_by='sip', color_by='dbyt_bucket', show_df_multiple=False, category_to_sm=category_sip_to_sm,
                      sm_type='xy', w=1600, h=1200,
                      sm_params={'x_field':'beg', 'y_field':'DBYT_median', 'line_groupby_field':'dbyt_bucket', 'line_groupby_w':2, 'dot_size':None, 'draw_labels':False}))

In [ ]:
_svg = rt.smallMultiples(df_dips_agg, category_by='dip', color_by='dbyt_bucket', show_df_multiple=False, category_to_sm=category_sip_to_sm,
                         sm_type='xy', w=1200, h=1000,
                         sm_params={'x_field':'beg', 'y_field':'DBYT_median', 'line_groupby_field':'dbyt_bucket', 'line_groupby_w':2, 'dot_size':None, 'draw_labels':False})
rt.displaySVG(_svg)

In [ ]:
rt.displaySVGAsImage(_svg)

In [ ]:
_svg = '''

<svg x="0" y="0" width="500" height="500">
  <rect x="0" y="0" width="500" height="500" fill="#ffffff" stroke="#0000ff" />

  <svg x="100" y="100" width="100" height="100">
    <rect x="0" y="0" width="100" height="100" fill-opacity="0" stroke="#ff0000" />
    <line x1="10" y1="10" x2="90" y2="90"  stroke="#000000" />
    <line x1="-50" y1="50" x2="50" y2="50" stroke="#00FF00" />
  </svg>

  <svg x="50" y="300" width="100" height="100">
    <rect x="0" y="0" width="100" height="100" fill-opacity="0" stroke="#ff0000" />
    <line x1="10" y1="10" x2="90" y2="90"  stroke="#000000" />
    <line x1="-50" y1="50" x2="50" y2="50" stroke="#00FF00" />
  </svg>

</svg>

'''
rt.displaySVG(_svg)

In [ ]:
rt.displaySVGAsImage(_svg)

In [ ]:
sampled_ip_to_svg = {}
i = 0
for k in ip_to_svg.keys():
    if (i%2) == 1:
        sampled_ip_to_svg[k] = ip_to_svg[k]
    i += 1
rt.displaySVG(rt.linkNode(df_dips, relates, pos, bounds_percent=0.1, draw_labels=True, link_shape='curve', link_arrow=True, node_shape=sampled_ip_to_svg, w=1600, h=1600))